# Deriving the Naive Bayes classifier:\n
   
  ## Applying Bayes Rule for P-1:
  
  P(Y=y_{k} | X=x_{0} ... x_{p-1} ) = \frac{P(X=x|Y=y_k)P(Y=y_k)}{P(X=x)}
    
    "\color{red}x"
    
   

In [1]:
import numpy.random as nprand
import math as math
import numpy as np
import random as rnd
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from random import randint


def bin_choose(c, theta_0, theta_1):
    if c == 0:
        return theta_0
    else:
        return theta_1
    
def generate_data(coin1,coin2):
    #the number of runs we do for each step
    data_runs_per_interval = 500
    #for i in range(1,data_runs_per_interval):
    theta_c0 = 0.6
    theta_c1 = 1 - theta_c0
    Coins = nprand.binomial([1 for i in range (data_runs_per_interval)], theta_c1)

    theta_h1_c0 = 0.3
    theta_h0_c0 = 1 - theta_h1_c0

    theta_h1_c1 = 0.4
    theta_h0_c1 = 1 - theta_h1_c1
    Hats = np.array([nprand.binomial(1, bin_choose(c, theta_h1_c0, theta_h1_c1)) for c in Coins])

    theta_f1_c0 = coin1
    theta_f0_c0 = 1.00 - theta_f1_c0

    theta_f1_c1 = coin2
    theta_f0_c1 = 1.00 - theta_f1_c1
    Flips = np.array([nprand.binomial(1, bin_choose(c, theta_f1_c0, theta_f1_c1)) for c in Coins])

    data = {'Flip' : [x for x in Flips], 
    'Hat'  : [x for x in Hats], 
    'Coin' : Coins}
    coindata = pd.DataFrame(data)
    return coindata


def calculate_accuracy(data_frame):
    clf = BernoulliNB()

    features = data_frame[['Flip', 'Hat']]
    labels = data_frame['Coin']
    clf.fit(features, labels)
    
   # theta_coin = [math.exp(x) for x in clf.class_log_prior_]  
    #feature_parameters = np.exp(clf.feature_log_prob_)
    #cvs = cross_val_score(clf, features, labels, cv=10)
    #print(cvs)
    #print(np.mean(cvs))

    #print("Accuracy", accuracy_score(labels, clf.predict(features)))
    #print("Accuracy", accuracy_score(labels, clf.predict(features)))
    return accuracy_score(labels,clf.predict(features))

def set_missing_data(data, category):
    amount_left = 0
    for i in range(len(data)):
        #print("go")
        random_value = randint(1,3)
        if(int(amount_left + random_value) >= len(data)):
            break
        data.at[(amount_left + random_value),category] = 50
        amount_left = amount_left + random_value
    return data

def set_random_data(data,category):
    for i in range(len(data)):
        #print("go")
        random_value = randint(0,1)
        if data.at[i,category] == 50:
            data.at[i,category] = random_value
    return data
        






#clean_data.to_csv("clean_data.csv")
#missing_data.to_csv("missing_data.csv")
#print(missing_data2.to_string())

    




In [2]:
#missing_data.describe()

In [ ]:
#clean_data.describe()

In [ ]:
N = 200

def create_lists():
    N = 100
    distance_increment = 0.000
    clean_list = []
    random_list = []
    for i in range(0,int(N/2)):
        coin1 = min(1,0.5 + distance_increment)
        coin2 = max(0,1 - coin1)

        #coin1 = round(coin1,3)
        #coin2 = round(coin2,3)

        #print(coin1)

        clean_data = generate_data(0.4,0.6)

        missing_data = generate_data(0.4,0.6)
        missing_data2 = set_missing_data(missing_data,'Hat')
        randomized_missing_data = set_random_data(missing_data2,'Hat')

        #distance = coin1 - coin2
        #print("accuracy is " + str(myScore))
        #print("distance is" + str(distance))
        acc1 = calculate_accuracy(clean_data)
        acc2 = calculate_accuracy(randomized_missing_data)
        
        clean_list.append(acc1)
        random_list.append(acc2)
        
        distance_increment += (1/N)
        #print(distance_increment)
    return clean_list,random_list


#clean_data_array,dirty_data_array = get_accuracy_arrays(clean_data,randomized_missing_data)
#print(clean_data_array)
#print(dirty_data_array)

import matplotlib.pyplot as plt
c_data,r_data= create_lists()
#print(c_data)
#print('----------')
#print(r_data)
plt.plot(c_data,'b')
plt.plot(r_data,'g')
#plt.ylabel('cleanacc')
#plt.xlabel('randomacc')
plt.show()



In [ ]:
def diff_accuracies(i1,i2):
    diff_acc_array = []
    for i in range(len(i1)):
        diff_acc = i1[i]-i2[i]
        diff_acc_array.append(diff_acc)
    return diff_acc_array
diff_accs = diff_accuracies(r_data,c_data)
plt.plot(diff_accs)
plt.show()

### Part C: Using the most likely value to make decisions (mean)

In [ ]:

likely_miss = generate_data(0.4,0.6)
likely_miss_set = set_missing_data(missing_data,'Hat')
def get_likely_value(data,category):
    likely_array = []
    for i in range(len(data)):
        #print("go")
        if data.at[i,category] != 50:
            likely_array.append(data.at[i,category])
    most_likely =  sum(likely_array)/len(likely_array)      
    return most_likely

def get_likely_dataset(data,category):
    likely = get_likely_value(data,category)
    for i in range(len(data)):
        if(likely < 0.5):
            data.at[i,category] = 0
        if(likely >= 0.5):
            data.at[i,category] = 1
    return data
    
#get_likely_dataset(likely_miss_set,'Hat');

def create_lists2():
    N = 100
    distance_increment = 0.000
    clean_list = []
    random_list = []
    for i in range(0,int(N/2)):
        coin1 = min(1,0.5 + distance_increment)
        coin2 = max(0,1 - coin1)

        #coin1 = round(coin1,3)
        #coin2 = round(coin2,3)

        #print(coin1)

        clean_data = generate_data(0.4,0.6)

        missing_data = generate_data(0.4,0.6)
        missing_data2 = set_missing_data(missing_data,'Hat')
        likely_data = get_likely_dataset(missing_data2,'Hat')

        #distance = coin1 - coin2
        #print("accuracy is " + str(myScore))
        #print("distance is" + str(distance))
        acc1 = calculate_accuracy(clean_data)
        acc2 = calculate_accuracy(likely_data)
        
        clean_list.append(acc1)
        random_list.append(acc2)
        
        distance_increment += (1/N)
        #print(distance_increment)
    return clean_list,random_list

c_data,l_data = create_lists2()

plt.plot(c_data,'b')
plt.plot(l_data,'g')
#plt.ylabel('cleanacc')
#plt.xlabel('randomacc')
plt.show()



In [ ]:
diff_accs2 = diff_accuracies(l_data,c_data)
plt.plot(diff_accs)
plt.show()
